<a href="https://colab.research.google.com/github/yangxy/GPEN/blob/main/GPEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before start, make sure that you choose

*   Python>=3.6
*   PyTorch>=1.7.1
*   CUDA>=10.0
*   CUDA Driver>=440
*   GCC>=4.9

in the Runtime menu --> Change runtime type

First, we clone the repository.

In [1]:
# # clone the repository
# %cd /content
# !rm -rf GPEN
# !git clone https://github.com/yangxy/GPEN.git
# %cd GPEN

Then, we set up the environment. 

In [2]:
# # set up the environment 
# !pip install -r requirements.txt

Download the pre-trained models and put them in ./weights.

In [3]:
# # download our pre-trained models
# !wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1961116085&Signature=GlUNW6%2B8FxvxWmE9jKIZYOOciKQ%3D" -O weights/RetinaFace-R50.pth
# !wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-BFR-512.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1961116208&Signature=hBgvVvKVSNGeXqT8glG%2Bd2t2OKc%3D" -O weights/GPEN-BFR-512.pth
# !wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-BFR-256.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1961116259&Signature=kMGJLSHqnvzzzqwtjUVBgngzX2s%3D" -O weights/GPEN-BFR-256.pth
# !wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/realesrnet_x2.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1962694780&Signature=lI%2FolhA%2FyigiTRvoDIVbtMIyhjI%3D" -O weights/realesrnet_x2.pth
# !wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/ParseNet-latest.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1961116134&Signature=bnMwU1JogmNbARto6G%2B7iaJQCHs%3D" -O weights/ParseNet-latest.pth

Finally, run our model.

In [8]:
# enhance faces
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import __init_paths
from face_enhancement import FaceEnhancement

def display(img1, img2):
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1) 
    plt.title('Input image', fontsize=16)
    ax1.axis('off')
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('GPEN output', fontsize=16)
    ax2.axis('off')
    ax1.imshow(img1)
    ax2.imshow(img2)

if __name__=="__main__":    
    model = {'name':'GPEN-BFR-512', 'sr_model':'realesrnet', 'sr_scale': 2, 'size':512, 'channel_multiplier':2, 'narrow':1}
    #model = {'name':'GPEN-BFR-256', 'sr_model':'realesrnet', 'size':256, 'channel_multiplier':1, 'narrow':0.5}
    
    indir = 'examples/imgs'
    outdir = 'examples/outs-bfr'
    os.makedirs(outdir, exist_ok=True)

    faceenhancer = FaceEnhancement(in_size=model['size'], model=model['name'], use_sr=True, sr_model=model['sr_model'], sr_scale=model['sr_scale'], channel_multiplier=model['channel_multiplier'], narrow=model['narrow'])

#     file = os.path.join(indir, 'Solvay_conference_1927.png')
    file = os.path.join(indir, 'selfsimswap.png')
    filename = os.path.basename(file)
    img = cv2.imread(file, cv2.IMREAD_COLOR) # BGR
    #im = cv2.resize(im, (0,0), fx=2, fy=2) #optional

    img_out, orig_faces, enhanced_faces = faceenhancer.process(img, aligned=False)
    
    img = cv2.resize(img, img_out.shape[:2][::-1])
    cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'_COMP.jpg'), np.hstack((img, img_out)))
    cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'_GPEN.jpg'), img_out)
        
    for m, (ef, of) in enumerate(zip(enhanced_faces, orig_faces)):
        of = cv2.resize(of, ef.shape[:2])
        cv2.imwrite(os.path.join(outdir, '.'.join(filename.split('.')[:-1])+'_face%02d'%m+'.jpg'), np.hstack((of, ef)))
    
    display(img, img_out)

AttributeError: 'NoneType' object has no attribute 'astype'

Download the results if needed.

In [7]:
!nvidia-smi

Thu May 26 02:17:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:1B:00.0 Off |                  N/A |
| 39%   28C    P8    22W / 350W |  16396MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:1C:00.0 Off |                  N/A |
| 39%   